# 05. NLP 패러다임의 진화: 데이터는 어떻게 지능이 되는가?
### 과목: NLP Transfer Learning
---
**이 노트북 하나로 완벽하게 이해하기**
이 파일은 단순한 코드 모음이 아닙니다. **"데이터가 어떻게 처리되고, 분석되어, 활용되는지"** 그 흐름과 진화 과정을 이야기처럼 풀었습니다.

**핵심 포인트 4가지**
1.  **데이터 정제와 분석:** 각 시대별로 텍스트를 어떻게 숫자/의미로 바꿨는가?
2.  **차이점과 필요성:** 왜 이전 기술을 버리고 새로운 기술이 나왔는가?
3.  **활용:** 분석된 결과는 어떻게 사용되는가?
4.  **기술의 고도화:** 코드는 어떻게 변해왔는가?

---

### 🎬 실습 데이터: 아주 짧은 영화 리뷰
우리는 이 4개의 문장을 가지고 **"긍정(1)"인지 "부정(0)"인지** 맞추는 모델을 만들 겁니다.

In [ ]:
# 입력 데이터 (X): 컴퓨터가 읽어야 할 글자들
reviews = [
    "이 영화 진짜 재밌다 최고의 명작",      # 1. 긍정
    "스토리 전개가 너무 지루하고 하품만 나옴", # 2. 부정
    "배우들의 연기력이 정말 대단하다 압도적임", # 3. 긍정
    "돈 아깝다 시간이 너무 아까워 최악"      # 4. 부정
]

# 정답 데이터 (y): 컴퓨터가 맞춰야 할 목표
labels = [1, 0, 1, 0]

---
## 1️⃣ Python 시대: "사람이 규칙을 다 정해줄게" (Rule-based)

### 1. 데이터 정제와 분석 방식
- **정제 (Preprocessing):** 문장을 띄어쓰기 기준으로 단순히 자릅니다. (`split`)
- **분석 (Analysis):** 데이터를 학습하지 않습니다. 사람이 머릿속에 있는 '긍정 단어장'을 코드에 직접 적어넣습니다.

### 2. 차이점과 필요성
- **필요성:** 컴퓨터에게 복잡한 수학을 가르치기 전, 가장 직관적인 방법이었습니다.
- **한계:** 사람이 세상의 모든 단어를 다 코딩할 수 없습니다. (예: '핵노잼' 같은 신조어 등장 시 인식 불가)

### 3. 활용
- 간단한 키워드 필터링(욕설 탐지 등)에 여전히 쓰입니다.

In [ ]:
def python_rule_based(text):
    # [1] 데이터 정제 (Preprocessing)
    # 문장을 쪼갭니다. "이 영화 진짜" -> ['이', '영화', '진짜']
    words = text.split()
    
    # [2] 분석 (Modeling 대신 Rule Definition)
    # 사람이 직접 규칙을 만듭니다. (학습 과정 없음)
    pos_keywords = ['재밌', '최고', '대단', '압도']
    neg_keywords = ['지루', '하품', '아깝', '최악']
    
    score = 0
    for word in words:
        # 긍정 단어가 있으면 +1점
        for pos in pos_keywords:
            if pos in word: score += 1
        # 부정 단어가 있으면 -1점
        for neg in neg_keywords:
            if neg in word: score -= 1
            
    # [3] 활용 (Output)
    # 점수가 0보다 크면 긍정으로 판단
    return "긍정" if score > 0 else "부정"

# 결과 확인
print("=== [1] 파이썬 규칙 기반 ===")
for r in reviews:
    print(f"'{r}' -> {python_rule_based(r)}")

---
## 2️⃣ 머신러닝 시대: "통계로 확률을 계산하자" (Statistical ML)

### 1. 데이터 정제와 분석 방식
- **정제:** **`CountVectorizer`**를 도입합니다. 글자를 '단어 가방(Bag of Words)'에 넣고 빈도수를 셉니다.
- **분석:** **`Naive Bayes`** 같은 통계 모델을 씁니다. "'최고'라는 단어가 나올 때 긍정일 확률이 80%네?"라고 컴퓨터가 계산합니다.

### 2. 차이점과 필요성
- **진화:** 사람이 규칙을 만드는 건 한계가 있어서, **데이터에서 통계적 패턴을 스스로 찾게** 만들었습니다.
- **한계:** 단어의 **순서**를 무시합니다. "밥 먹고 배 아파"와 "배 아파 밥 먹고"를 똑같이 인식합니다.

### 3. 활용
- 스팸 메일 필터링, 뉴스 카테고리 분류 등에 아주 많이 쓰였습니다.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# [1] 데이터 정제 (Preprocessing)
# 이제 단어를 사람이 쪼개지 않습니다. Vectorizer가 '단어장'을 만들고 숫자로 바꿉니다.
vectorizer = CountVectorizer()
X_vec = vectorizer.fit_transform(reviews)

print("\n=== [2] 머신러닝 (통계 기반) ===")
print("컴퓨터가 만든 단어장:", vectorizer.get_feature_names_out())
print("숫자로 변한 문장(빈도수):", X_vec[0].toarray())

# [2] 분석 (Modeling)
# 규칙 대신, 데이터(X_vec)와 정답(labels)을 주고 확률을 학습시킵니다.
ml_model = MultinomialNB()
ml_model.fit(X_vec, labels)

# [3] 활용 (Inference)
# 새로운 문장이 들어오면 확률을 계산해서 답을 줍니다.
test_sent = ["영화 정말 최고"]
test_vec = vectorizer.transform(test_sent)
pred = ml_model.predict(test_vec)
print(f"활용 결과: '{test_sent[0]}' -> {'긍정' if pred[0]==1 else '부정'}")

---
## 3️⃣ 딥러닝 시대: "단어의 의미와 문맥을 이해하자" (Deep Learning)

### 1. 데이터 정제와 분석 방식
- **정제:** **`Tokenizer`**로 단어에 고유 번호를 붙이고, **`Embedding`**이라는 기술로 단어를 '의미 벡터'로 바꿉니다.
- **분석:** **`RNN/LSTM`** 신경망을 써서 문장을 앞에서부터 순서대로 읽습니다. 문맥(Conext)을 기억합니다.

### 2. 차이점과 필요성
- **진화:** 머신러닝이 놓쳤던 **순서와 뉘앙스**를 잡기 위해 신경망(뇌 구조)을 모방했습니다.
- **한계:** 학습 데이터가 엄청나게 많이 필요하고, 학습 시간이 오래 걸립니다.

### 3. 활용
- 번역기(파파고 초창기), 음성 인식, 챗봇 등에 혁명을 일으켰습니다.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import numpy as np

# [1] 데이터 정제 (Preprocessing)
# 문장을 숫자로 바꾸고, 길이를 똑같이 맞춥니다. (Padding)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews)
sequences = tokenizer.texts_to_sequences(reviews)
padded = pad_sequences(sequences, padding='post', maxlen=5)

print("\n=== [3] 딥러닝 (신경망) ===")
print("정제된 데이터(Sequence):\n", padded)

# [2] 분석 (Modeling)
# Embedding(의미 파악) -> LSTM(순서 파악) -> Dense(출력)
vocab_size = len(tokenizer.word_index) + 1
dl_model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=4, input_length=5),
    LSTM(8),
    Dense(1, activation='sigmoid')
])
dl_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
dl_model.fit(padded, np.array(labels), epochs=10, verbose=0)

# [3] 활용 (Inference)
# 0.5보다 크면 긍정으로 판단
pred = dl_model.predict(padded)[0][0]
print(f"활용 결과 (첫 문장): {pred:.4f} -> {'긍정' if pred>0.5 else '부정'}")

---
## 4️⃣ LLM 시대: "이미 다 아는 천재를 데려다 쓰자" (Transfer Learning)

### 1. 데이터 정제와 분석 방식
- **정제:** **`AutoTokenizer`**를 씁니다. BERT가 학습할 때 썼던 그 규칙 그대로 쪼개고, 특수 토큰(`[CLS]`)을 붙입니다.
- **분석:** **`Transformer(BERT)`** 모델을 씁니다. 처음부터 배우는 게 아니라, 이미 수십억 개의 문장을 읽은 모델을 가져와서 미세 조정합니다.

### 2. 차이점과 필요성
- **진화:** "처음부터 가르치지 말고, 잘하는 애(Pre-trained) 데려와서 내 일만 가르치자(Fine-tuning)"는 혁명적인 생각.
- **결과:** 적은 데이터로도 압도적인 성능을 냅니다.

### 3. 활용
- ChatGPT, 현재의 모든 AI 서비스, 고성능 문서 분류, 검색 엔진 등.

In [ ]:
from transformers import AutoTokenizer, TFBertForSequenceClassification
import tensorflow as tf

# 모델 이름 (한국어 BERT)
MODEL_NAME = "klue/bert-base"

# [1] 데이터 정제 (Preprocessing)
# 천재 모델이 사용하는 전용 도구(Tokenizer)를 그대로 가져옵니다.
# 알아서 [CLS], [SEP] 같은 특수 토큰을 붙이고 숫자로 만들어줍니다.
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
inputs = tokenizer(reviews, padding=True, truncation=True, return_tensors="tf")

print("\n=== [4] LLM (전이 학습) ===")
print("정제된 데이터 (BERT Input):\n", inputs['input_ids'].numpy()[0])

# [2] 분석 (Modeling)
# 모델의 '뇌(Weights)'를 그대로 다운로드 받습니다.
llm_model = TFBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2, from_pt=True)

# [3] 활용 (Inference)
# 별도의 학습 없이도 기본적인 언어 능력이 뛰어나서 바로 추론해봅니다.
outputs = llm_model(inputs)
probs = tf.nn.softmax(outputs.logits, axis=-1)
predictions = tf.argmax(probs, axis=-1)

print(f"활용 결과: {predictions.numpy()} (모델이 기본 지식으로 판단한 결과)")

## 🏆 총정리: 기술의 고도화 과정

| 시대 | 기술 이름 | 핵심 변화 (Why?) | 코드의 특징 |
|:---:|:---:|:---:|:---|
| **1** | **Python** | "규칙이 필요해" | `if`문 도배, 모든 단어를 사람이 입력 |
| **2** | **ML** | "규칙 못 짜겠어, 통계 내자" | `Vectorizer`, 확률 계산, 단어 가방(BoW) |
| **3** | **DL** | "문맥을 이해해야지!" | `Embedding`, `LSTM`, 순서와 뉘앙스 파악 |
| **4** | **LLM** | "그냥 천재 데려와" | `Pre-trained` 로드, 코드 양은 줄고 성능은 폭발 |

보시다시피 기술이 발전할수록 **"사람이 할 일은 줄어들고, 컴퓨터가 미리 해놓은 일(사전학습)을 가져다 쓰는 방식"**으로 고도화되었습니다.